# Trade Network Analysis
`09_generate_panel.ipynb`

---

METODO:
1. take log of series
2. Fix IMF series! include 2007
3. Drop unbalanced

#### Libraries

In [40]:
import pandas as pd                                         # pandas dataframes used for convenience
import os                                                   # change current directory in next line
os.chdir('C:/Working/trade_network/data/')

Load IMF data

In [66]:
decomp = pd.read_csv('imports_decomp.csv')
decomp = decomp.rename(columns={'Unnamed: 0': 'date', 'j': 'iso2'})
decomp['date'] = decomp['date'].apply(str)
decomp = decomp.set_index(['date', 'iso2'])
df = pd.read_csv('imf_data.csv', index_col=[0,1,2,3], skipinitialspace=True).reset_index()[2:]
df = df.rename(columns={'level_0': 'iso2', 'Unnamed: 1': 'full_name', 'Unnamed: 2':'date', 'mp': 'mp_base', 
                        'mp.1': 'mp',  'xp': 'xp_base', 'xp.1': 'xp'})
df = df.set_index(['date', 'iso2'])
panel = df.join(decomp)
for v in ['q', 'x', 'xp', 'mp']:
    panel[v] = panel[v].apply(float) # replace country names

#### Deflate USD values with price indices

In [70]:
deflators = {'x': 'xp', 'cs': 'mp', 'ns': 'mp', 'cn': 'mp', 'nn': 'mp'}
for k, v in deflators.iteritems():
    panel[k+'_def'] = (panel[k] * (panel[v] / 100))

In [79]:
panel['x_ch'] = panel['x_def'].groupby(df.index.get_level_values(1)).diff()

In [82]:
#panel.dropna()[['q', 'x_def', 'cs_def', 'ns_def', 'cn_def', 'x_ch']]
from pandas.stats.plm import PanelOLS

In [86]:
reg  = PanelOLS(y=panel['x_ch'],x=panel[['x_def', 'q', 'cs_def', 'ns_def', 'cn_def']],time_effects=True)
reg


-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <x_def> + <q> + <cs_def> + <ns_def> + <cn_def>

Number of Observations:         523
Number of Degrees of Freedom:   11

R-squared:         0.1350
Adj R-squared:     0.1181

Rmse:          56604553707.5237

F-stat (5, 512):     7.9876, p-value:     0.0000

Degrees of Freedom: model 10, resid 512

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
         x_def     0.0323     0.0222       1.46     0.1462    -0.0112     0.0758
             q 582558890.0231 366943309.7502       1.59     0.1130 -136649997.0874 1301767777.1336
        cs_def  1367.7285   548.4123       2.49     0.0129   292.8403  2442.6166
        ns_def  -361.3403   164.2761      -2.20     0.0283  -683.3214   -39.3592
        cn_def    76.6